- in order to get analyzed but not normalized dataset.
- delete chart code to decreate running time

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from sklearn.preprocessing import normalize
from sklearn.impute import SimpleImputer
from sklearn import linear_model
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression

import statsmodels.api as sm

import scipy

## load data

In [3]:
plt.rcParams["figure.figsize"] = (15, 10)

In [4]:
pth_data = '../data/train.csv'
df_origin = pd.read_csv(pth_data)

In [5]:
df_origin

,time,height,weight,gender,bmi,age,env_temp,rh,heart_rate,stress_level,skin_temp,eda,TC,TS,Clo,Act
0,2020-10-19 14:00:00,1.64,54.0,1,20.077335,24,77.990,62.122,NaN,17.0,NaN,NaN,4.0,4.0,0.74,1.0
1,2020-10-19 14:30:00,1.64,54.0,1,20.077335,24,78.208,62.321,NaN,60.0,30.748,0.926320,4.0,4.0,0.74,1.1
2,2020-10-19 15:00:00,1.64,54.0,1,20.077335,24,78.514,62.255,NaN,46.0,31.327,1.232405,4.0,4.0,0.74,1.1
3,2020-10-19 15:30:00,1.64,54.0,1,20.077335,24,79.041,61.491,NaN,91.0,31.458,0.375005,4.0,4.0,0.74,1.0
4,2020-10-19 16:00:00,1.64,54.0,1,20.077335,24,79.435,61.071,NaN,71.0,32.085,0.081127,4.0,4.0,0.74,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2791,2020-10-28 21:30:00,1.63,50.0,1,18.818924,23,76.528,40.530,73.0,22.0,31.850,0.114876,2.0,3.0,0.50,1.0
2792,2020-10-28 22:00:00,1.63,50.0,1,18.818924,23,76.876,39.890,NaN,10.0,32.215,0.146231,2.0,3.0,0.50,1.0
2793,2020-10-28 22:30:00,1.63,50.0,1,18.818924,23,77.050,39.710,NaN,5.0,32.656,0.153838,2.0,3.0,0.50,1.0
2794,2020-10-28 23:00:00,1.63,50.0,1,18.818924,23,76.703,39.980,NaN,0.0,31.902,0.160930,2.0,3.0,0.50,1.0


In [6]:
df_origin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2796 entries, 0 to 2795
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   time          2796 non-null   object 
 1   height        2796 non-null   float64
 2   weight        2796 non-null   float64
 3   gender        2796 non-null   int64  
 4   bmi           2796 non-null   float64
 5   age           2796 non-null   int64  
 6   env_temp      2796 non-null   float64
 7   rh            2796 non-null   float64
 8   heart_rate    1669 non-null   float64
 9   stress_level  1848 non-null   float64
 10  skin_temp     1874 non-null   float64
 11  eda           2396 non-null   float64
 12  TC            2717 non-null   float64
 13  TS            2712 non-null   float64
 14  Clo           2737 non-null   float64
 15  Act           2718 non-null   float64
dtypes: float64(13), int64(2), object(1)
memory usage: 349.6+ KB


## data process

### choose data: TS >= 3.0

In [7]:
df_origin[df_origin[['TS']].isna().TS].shape
# some na in clo, means unrecorded data

(84, 16)

In [8]:
df_record = df_origin[df_origin['TS'].isna() ^ True]

In [9]:
df_record.shape

(2712, 16)

In [10]:
df_comfort = df_record[df_record['TC'] >= 3.0]

### shuffle

In [11]:
df_shuffle = df_comfort.sample(frac=1, random_state=1208).reset_index(drop=True)

In [12]:
X_data = df_shuffle[['height', 'weight', 'gender', 'bmi', 'age', 'rh', \
                      'heart_rate', 'stress_level', 'skin_temp', 'eda', \
                      'TC', 'TS', 'Clo', 'Act']]

In [13]:
X_cols_name = X_data.columns

In [14]:
y_data = df_shuffle['env_temp']

### imputation

In [15]:
df_record[df_record[['env_temp']].isna().env_temp]
# no na in env_temp

,time,height,weight,gender,bmi,age,env_temp,rh,heart_rate,stress_level,skin_temp,eda,TC,TS,Clo,Act


In [16]:
imp = SimpleImputer(strategy='mean')

In [17]:
X_data = pd.DataFrame(imp.fit_transform(X_data), columns=X_cols_name)

In [18]:
X_data_not_norm = pd.DataFrame(imp.fit_transform(X_data), columns=X_cols_name)

### normalize

In [19]:
X_array_normalize, X_norm = normalize(X_data, axis=0, return_norm=True)

In [20]:
X_data = pd.DataFrame(X_array_normalize, columns=X_cols_name)

In [21]:
X_data

,height,weight,gender,bmi,age,rh,heart_rate,stress_level,skin_temp,eda,TC,TS,Clo,Act
0,0.018674,0.016305,0.019889,0.018573,0.019347,0.024127,0.020524,0.010107,0.019882,0.000866,0.020182,0.018018,0.022576,0.018588
1,0.020659,0.018013,-0.019889,0.016766,0.018541,0.018656,0.019535,0.001037,0.019217,0.000190,0.020182,0.018018,0.013323,0.018588
2,0.019958,0.019877,-0.019889,0.019822,0.021765,0.018383,0.019618,0.017104,0.020624,0.002329,0.020182,0.018018,0.022576,0.016898
3,0.020659,0.018013,-0.019889,0.016766,0.018541,0.020090,0.022750,0.027470,0.018905,0.000048,0.020182,0.018018,0.013323,0.016898
4,0.020659,0.018013,-0.019889,0.016766,0.018541,0.021603,0.019618,0.041464,0.018691,0.000018,0.020182,0.018018,0.013323,0.016898
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2523,0.019375,0.017703,0.019889,0.018733,0.020153,0.020279,0.019618,0.001037,0.019168,0.000043,0.020182,0.018018,0.019985,0.016898
2524,0.019725,0.023293,-0.019889,0.023782,0.019347,0.018008,0.019618,0.016643,0.019445,0.000125,0.015137,0.018018,0.013323,0.016898
2525,0.018674,0.019566,0.019889,0.022287,0.019347,0.017529,0.025964,0.016643,0.019882,0.000064,0.020182,0.018018,0.018505,0.018588
2526,0.020308,0.020188,-0.019889,0.019443,0.018541,0.020548,0.019618,0.016643,0.021215,0.000643,0.020182,0.031532,0.019985,0.013519


In [22]:
X_norm

array([  85.67889413, 3219.80907819,   50.27922036, 1104.20171125,
       1240.52690418, 2610.52082164, 4044.03947436, 1929.40521588,
       1609.69440491,  204.39639127,  198.19434906,  221.99549545,
         27.02021465,   59.17786748])

## refine data

In [23]:
regr = sm.OLS(y_data, X_data)
regr_fit = regr.fit()

### leverage & outlier

In [24]:
infl = regr_fit.get_influence()

In [25]:
# leverage
leviers = infl.hat_matrix_diag
# list(leviers)

In [26]:
# studentized residue
res_studs = infl.resid_studentized_external
# list(res_studs)

### leverage detection
the threshold value may be:
$$s_h=2\times \frac{p+1}{n}$$

In [27]:
n_data = X_data.shape[0]
p_data = X_data.shape[1]

In [28]:
s_h = 2 * (p_data + 1) / n_data
s_h

0.011867088607594937

In [29]:
atyp_levier = leviers > s_h

In [30]:
len(leviers[atyp_levier])

113

### outlier detection
Threshold:
$$s_t=t_{1-0.05/2}(n-p-2)$$

In [31]:
seuil_stud = scipy.stats.t.ppf(0.975, df=n_data-p_data-2)
seuil_stud

1.9609088074865402

In [32]:
atyp_stud = np.abs(res_studs) > seuil_stud
atyp_stud

array([False, False, False, ...,  True, False,  True])

In [33]:
idx_eliminate_lvg_outl = np.logical_not(np.logical_or(atyp_levier, atyp_stud))

In [34]:
X_data.index[idx_eliminate_lvg_outl]

Int64Index([   0,    1,    2,    3,    4,    5,    7,    8,    9,   10,
            ...
            2516, 2517, 2518, 2519, 2520, 2521, 2522, 2523, 2524, 2526],
           dtype='int64', length=2287)

In [35]:
X_data_refine = X_data[idx_eliminate_lvg_outl]
y_data_refine = y_data[idx_eliminate_lvg_outl]

### save refine data (not normalized)

In [36]:
X_data_refine_not_norm = X_data_not_norm[idx_eliminate_lvg_outl]
y_data_refine_not_norm = y_data[idx_eliminate_lvg_outl]

In [37]:
df_refine_not_norm = pd.concat([X_data_refine_not_norm, y_data_refine_not_norm], axis=1)

In [38]:
df_refine_not_norm.to_csv('./processed_data/train_refine_not_norm.csv', index=False)

## feature selection

### backward selection

In [39]:
regr = sm.OLS(y_data_refine, X_data_refine)
regr_fit = regr.fit()

In [40]:
regr_fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:               env_temp   R-squared (uncentered):                   0.999
Model:                            OLS   Adj. R-squared (uncentered):              0.999
Method:                 Least Squares   F-statistic:                          2.758e+05
Date:                Thu, 12 Nov 2020   Prob (F-statistic):                        0.00
Time:                        13:24:22   Log-Likelihood:                         -4675.0
No. Observations:                2287   AIC:                                      9378.
Df Residuals:                    2273   BIC:                                      9458.
Df Model:                          14                                                  
Covariance Type:            nonrobust                                                  
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
height        3120.3694     86.916     35.901      0.000    2949.926    3290.813
weight       -1569.3440     68.293    -22.980      0.000   -1703.267   -1435.421
gender           9.4058      4.641      2.027      0.043       0.305      18.507
bmi           1654.9435     63.034     26.255      0.000    1531.332    1778.555
age            -29.8858     30.888     -0.968      0.333     -90.457      30.686
rh              28.5008     12.745      2.236      0.025       3.509      53.493
heart_rate     -13.6848     18.635     -0.734      0.463     -50.228      22.858
stress_level    23.9767      4.271      5.614      0.000      15.602      32.351
skin_temp      547.5106     77.583      7.057      0.000     395.370     699.651
eda              3.2009      7.075      0.452      0.651     -10.673      17.075
TC             109.9957     13.043      8.433      0.000      84.418     135.574
TS             129.6056     11.204     11.568      0.000     107.634     151.577
Clo           -121.1534      6.887    -17.592      0.000    -134.659    -107.648
Act             -9.2039      9.658     -0.953      0.341     -28.143       9.735
==============================================================================
Omnibus:                        5.849   Durbin-Watson:                   2.008
Prob(Omnibus):                  0.054   Jarque-Bera (JB):                5.853
Skew:                           0.110   Prob(JB):                       0.0536
Kurtosis:                       2.885   Cond. No.                         224.
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [41]:
X_rm = X_data_refine

start to remove features

#### remove eda

In [42]:
X_rm = X_rm.drop(columns='eda')

In [43]:
regr = sm.OLS(y_data_refine, X_rm)
regr_fit = regr.fit()
regr_fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:               env_temp   R-squared (uncentered):                   0.999
Model:                            OLS   Adj. R-squared (uncentered):              0.999
Method:                 Least Squares   F-statistic:                          2.971e+05
Date:                Thu, 12 Nov 2020   Prob (F-statistic):                        0.00
Time:                        13:24:38   Log-Likelihood:                         -4675.1
No. Observations:                2287   AIC:                                      9376.
Df Residuals:                    2274   BIC:                                      9451.
Df Model:                          13                                                  
Covariance Type:            nonrobust                                                  
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
height        3116.1037     86.388     36.071      0.000    2946.695    3285.512
weight       -1565.6111     67.781    -23.098      0.000   -1698.530   -1432.692
gender           9.5756      4.625      2.070      0.039       0.506      18.645
bmi           1651.6986     62.614     26.379      0.000    1528.912    1774.486
age            -29.5383     30.873     -0.957      0.339     -90.081      31.004
rh              28.5872     12.741      2.244      0.025       3.602      53.572
heart_rate     -13.6153     18.631     -0.731      0.465     -50.150      22.920
stress_level    24.0814      4.264      5.648      0.000      15.721      32.442
skin_temp      550.6926     77.250      7.129      0.000     399.205     702.180
TC             110.3729     13.014      8.481      0.000      84.852     135.894
TS             129.4988     11.200     11.563      0.000     107.536     151.462
Clo           -121.1581      6.886    -17.596      0.000    -134.661    -107.655
Act             -9.1049      9.654     -0.943      0.346     -28.036       9.826
==============================================================================
Omnibus:                        5.890   Durbin-Watson:                   2.008
Prob(Omnibus):                  0.053   Jarque-Bera (JB):                5.894
Skew:                           0.110   Prob(JB):                       0.0525
Kurtosis:                       2.885   Cond. No.                         222.
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

#### remove heart rate

In [44]:
X_rm = X_rm.drop(columns='heart_rate')

In [45]:
regr = sm.OLS(y_data_refine, X_rm)
regr_fit = regr.fit()
regr_fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:               env_temp   R-squared (uncentered):                   0.999
Model:                            OLS   Adj. R-squared (uncentered):              0.999
Method:                 Least Squares   F-statistic:                          3.219e+05
Date:                Thu, 12 Nov 2020   Prob (F-statistic):                        0.00
Time:                        13:24:43   Log-Likelihood:                         -4675.3
No. Observations:                2287   AIC:                                      9375.
Df Residuals:                    2275   BIC:                                      9443.
Df Model:                          12                                                  
Covariance Type:            nonrobust                                                  
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
height        3096.7109     82.203     37.671      0.000    2935.510    3257.912
weight       -1549.5626     64.118    -24.167      0.000   -1675.299   -1423.826
gender          10.2071      4.543      2.247      0.025       1.298      19.116
bmi           1637.3556     59.453     27.541      0.000    1520.769    1753.943
age            -28.9661     30.860     -0.939      0.348     -89.483      31.550
rh              28.4976     12.739      2.237      0.025       3.516      53.479
stress_level    22.6198      3.765      6.008      0.000      15.237      30.003
skin_temp      556.1049     76.886      7.233      0.000     405.331     706.879
TC             109.9905     13.002      8.459      0.000      84.493     135.488
TS             130.1154     11.167     11.652      0.000     108.217     152.014
Clo           -121.0909      6.884    -17.589      0.000    -134.591    -107.591
Act             -9.7869      9.608     -1.019      0.308     -28.627       9.053
==============================================================================
Omnibus:                        6.138   Durbin-Watson:                   2.009
Prob(Omnibus):                  0.046   Jarque-Bera (JB):                6.128
Skew:                           0.112   Prob(JB):                       0.0467
Kurtosis:                       2.880   Cond. No.                         203.
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

#### remove age

In [46]:
X_rm = X_rm.drop(columns='age')

In [47]:
regr = sm.OLS(y_data_refine, X_rm)
regr_fit = regr.fit()
regr_fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:               env_temp   R-squared (uncentered):                   0.999
Model:                            OLS   Adj. R-squared (uncentered):              0.999
Method:                 Least Squares   F-statistic:                          3.512e+05
Date:                Thu, 12 Nov 2020   Prob (F-statistic):                        0.00
Time:                        13:24:59   Log-Likelihood:                         -4675.8
No. Observations:                2287   AIC:                                      9374.
Df Residuals:                    2276   BIC:                                      9437.
Df Model:                          11                                                  
Covariance Type:            nonrobust                                                  
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
height        3076.0354     79.195     38.841      0.000    2920.733    3231.337
weight       -1554.2272     63.924    -24.314      0.000   -1679.582   -1428.872
gender           9.3212      4.444      2.098      0.036       0.607      18.036
bmi           1633.3052     59.294     27.546      0.000    1517.029    1749.582
rh              29.1435     12.720      2.291      0.022       4.199      54.088
stress_level    23.2577      3.703      6.281      0.000      15.996      30.519
skin_temp      557.8901     76.860      7.258      0.000     407.166     708.614
TC             107.3735     12.700      8.455      0.000      82.469     132.278
TS             130.0551     11.166     11.647      0.000     108.158     151.952
Clo           -121.1574      6.884    -17.600      0.000    -134.657    -107.658
Act             -9.7012      9.607     -1.010      0.313     -28.540       9.138
==============================================================================
Omnibus:                        5.696   Durbin-Watson:                   2.009
Prob(Omnibus):                  0.058   Jarque-Bera (JB):                5.670
Skew:                           0.105   Prob(JB):                       0.0587
Kurtosis:                       2.877   Cond. No.                         192.
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

#### remove Act

In [48]:
X_rm = X_rm.drop(columns='Act')

In [49]:
regr = sm.OLS(y_data_refine, X_rm)
regr_fit = regr.fit()
regr_fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:               env_temp   R-squared (uncentered):                   0.999
Model:                            OLS   Adj. R-squared (uncentered):              0.999
Method:                 Least Squares   F-statistic:                          3.863e+05
Date:                Thu, 12 Nov 2020   Prob (F-statistic):                        0.00
Time:                        13:25:06   Log-Likelihood:                         -4676.3
No. Observations:                2287   AIC:                                      9373.
Df Residuals:                    2277   BIC:                                      9430.
Df Model:                          10                                                  
Covariance Type:            nonrobust                                                  
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
height        3063.3454     78.192     39.177      0.000    2910.010    3216.680
weight       -1552.1393     63.891    -24.294      0.000   -1677.429   -1426.849
gender           8.7978      4.414      1.993      0.046       0.143      17.453
bmi           1629.4979     59.174     27.537      0.000    1513.456    1745.539
rh              29.0763     12.720      2.286      0.022       4.132      54.020
stress_level    22.7955      3.675      6.203      0.000      15.589      30.001
skin_temp      564.0294     76.620      7.361      0.000     413.777     714.282
TC             107.7202     12.695      8.485      0.000      82.825     132.615
TS             130.0303     11.166     11.645      0.000     108.133     151.928
Clo           -121.6697      6.865    -17.723      0.000    -135.132    -108.207
==============================================================================
Omnibus:                        5.700   Durbin-Watson:                   2.008
Prob(Omnibus):                  0.058   Jarque-Bera (JB):                5.672
Skew:                           0.105   Prob(JB):                       0.0587
Kurtosis:                       2.876   Cond. No.                         182.
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### save selected data (not normalized)

In [51]:
X_selected = X_data_refine_not_norm.drop(columns=['eda', 'heart_rate', 'age', 'Act'])
y_selected = y_data_refine_not_norm

In [52]:
df_selected_not_norm = pd.concat([X_selected, y_selected], axis=1)

In [54]:
df_selected_not_norm.to_csv('./processed_data/train_selected_not_norm.csv', index=False)